First of all, I want to import all packages that I need for the project.

In [295]:
import csv
import kagglehub
import os

We need to analyse the dataset to see, how the dataset looks like.

In [296]:
path_google_dataset = kagglehub.dataset_download("lava18/google-play-store-apps")
print("Path to dataset files:", path_google_dataset)
path_ios_dataset = kagglehub.dataset_download("ramamet4/app-store-apple-data-set-10k-apps")
print("Path to dataset files:", path_ios_dataset)

Path to dataset files: /Users/marekpapay/.cache/kagglehub/datasets/lava18/google-play-store-apps/versions/6
Path to dataset files: /Users/marekpapay/.cache/kagglehub/datasets/ramamet4/app-store-apple-data-set-10k-apps/versions/7


In [297]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

I will open the datasets.

In [298]:
google_dataset_path = "/Users/marekpapay/.cache/kagglehub/datasets/lava18/google-play-store-apps/versions/6/googleplaystore.csv" 
apple_dataset_path = "/Users/marekpapay/.cache/kagglehub/datasets/ramamet4/app-store-apple-data-set-10k-apps/versions/7/AppleStore.csv"

In [299]:
with open(apple_dataset_path, newline='') as ios_file: 
    dataset_ios = list(csv.reader(ios_file)) 
with open(google_dataset_path, newline='') as google_file: 
    dataset_google = list(csv.reader(google_file))

In [ ]:
for x in dataset_google:
    print(x[7])

I want to analyse it and see the headers of the datasets and number of rows and number of columns

In [300]:
explore_data(dataset_ios, 0, 1, True)
print('\n')
explore_data(dataset_google, 0, 1, True)

['', 'id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


Number of rows: 7198
Number of columns: 17


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


Number of rows: 10842
Number of columns: 13


* Below, I want to print the most important columns, that I will focus on in the future analysis.

In [301]:
print('Apple:' 'track_name', 'rating_count_tot', 'rating_count_ver', 'user_rating',
     'user_rating_ver')
print('Google:' 'Category','Rating', 'Type',  )

Apple:track_name rating_count_tot rating_count_ver user_rating user_rating_ver
Google:Category Rating Type


I will write a function that deletes rows, that are wrong in the dataset. Those rows are said to be wrong in the forum of the dataset in Kaggle.

In [302]:
def delete_wrong_rows(dataset: list, index_number: int, start_number: int):
    #I want to put index for all the rows
    for index, row in enumerate(dataset, start = index_number):
        #I want to see if there is a match
        if index == index_number:
            #delete the row
            del dataset[index]

In [303]:
delete_wrong_rows(dataset_google, 10473, 0)

As there are apps that are duplicated, I need to create a function, that will delete all the duplicities.

In [304]:
def process_apps(dataset: list, app_name: int, newest_app: int):
    duplicate_apps_google = []
    unique_apps_google = []

    # Identify duplicate and unique apps
    for app in dataset:
        name = app[app_name]
        if name in [app[app_name] for app in unique_apps_google]:
            duplicate_apps_google.append(app)
        else:
            unique_apps_google.append(app)  

    # Find the newest app version in duplicates
    clear_newest_App_in_Dataset = {}

    for highest_app in duplicate_apps_google:
        app = str(highest_app[app_name])
        number_app = int(highest_app[newest_app])
        
        if app not in clear_newest_App_in_Dataset:
            clear_newest_App_in_Dataset[app] = number_app
        
        elif clear_newest_App_in_Dataset[app] < number_app:
            clear_newest_App_in_Dataset[app] = number_app

    # Create a list of unique apps with the newest version
    for index, app in enumerate(dataset):
        application = app[app_name]
        number_app = app[newest_app]

        for key, value in clear_newest_App_in_Dataset.items():

            if application == key and number_app == value and number_app not in (dataset[app_name]):
                unique_apps_google.append(app)
    return unique_apps_google

In [312]:
google_dataset = process_apps(dataset_google, 0, 3)
dataset_ios = process_apps(dataset_ios, 2, 3)
print(len(google_dataset))
print(len(dataset_ios))

9660
7196


We want to keep only the apps that are written in english, all others want to delete. We want to keep only ASCII.

In [314]:
def delete_non_english_apps(dataset: list, index: int):
    english_apps = []
    
    for row in dataset:
        word = row[index]  # Extract the word
        # Check if all characters are English and the app name is not already in the list
        if all(ord(letter) < 128 for letter in word) and row not in english_apps:
            english_apps.append(row)
    
    return english_apps

In [315]:
google_dataset = delete_non_english_apps(google_dataset, 0)
ios_dataset = delete_non_english_apps(dataset_ios, 2)

I want to isolate it only on Free Apps

In [ ]:
def only_free_apps(dataset: str, index: int):
    free_apps = []
    non_free_apps = []
    for row in dataset:
        if row[index] == 0:
            free_apps.append(row)
        else:
            non_free_apps.append(row)

In [ ]:
free_apps_ios = 
paid_apps_ios =
